### Caffe Model-나이/성별 예측

In [3]:
# ! pip install dlib

In [1]:
# caffemodel 다운로드 URL : https://github.com/habom2310/People-tracking-with-Age-and-Gender-detection

import glob
import dlib
import cv2

ModuleNotFoundError: No module named 'dlib'

In [ ]:
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']  # 나이 분류 구간 정의
gender_list = ['Male','Female']  # 성별 구분 정의

In [ ]:
# 얼굴을 감지하는 detector
detector = dlib.get_frontal_face_detector()      

# 나이를 감지하는 detector
age_detector = cv2.dnn.readNetFromCaffe(          
               'c:/data/model/deploy_age.prototxt',
               'c:/data/model/age_net.caffemodel')

# 성별을 감지하는 detector        
gender_detector = cv2.dnn.readNetFromCaffe(      
               'c:/data/model/deploy_gender.prototxt',
               'c:/data/model/gender_net.caffemodel')

img_list = glob.glob('c:/workspace3/data/images/age_gender/*.jpg')  

In [ ]:
len(img_list)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for img_path in img_list:
  image = cv2.imread(img_path) 
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  faces = detector(gray, 1)  # detector 에 의해 얼굴 감지
  print("Number of faces detected: {}".format(len(faces)))

  for face in faces:
    # boxing 좌표
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()  

    # 이미지에서 얼굴 영역만 copy
    face_img = image[y1:y2, x1:x2].copy()  
    blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),
           mean=(78.4263377603, 87.7689143744, 114.895847746), swapRB=False, crop=False)

    # predict age
    age_detector.setInput(blob)
    age_preds = age_detector.forward()
    age = age_list[age_preds[0].argmax()]

    # predict gender
    gender_detector.setInput(blob)
    gender_preds = gender_detector.forward()
    gender = gender_list[gender_preds[0].argmax()]

    cv2.rectangle(image, (x1, y1), (x2, y2), (255,255,255), 2)

    overlay_text = '%s %s' % (gender, age)

    cv2.putText(image, overlay_text, org=(x1, y1), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0,0,0), thickness=10)

    cv2.putText(image, overlay_text, org=(x1, y1),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255), thickness=2)

  image=cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
  plt.figure()
  plt.imshow(image)

  plt.show()  
  
import cv2 as cv
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions

model=ResNet50(weights='imagenet')
img=cv.imread('d:/images/rabbit.jpg')

x=np.reshape(cv.resize(img,(224,224)),(1,224,224,3))  
x=preprocess_input(x)
preds=model.predict(x)

top5=decode_predictions(preds,top=5)[0]
print('예측 결과:',top5)

for i in range(5):
    cv.putText(img,top5[i][1]+':'+str(top5[i][2]),(10,20+i*20),cv.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)

cv.imshow('Recognition result',img)
cv.waitKey()
cv.destroyAllWindows()